In [2]:
import numpy as np
from numpy import array
import math

import pandas as pd 

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
#ENTER THE UAH DRIVESET LOCATION
path = 'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/UAH-DRIVESET-v1/'

In [3]:
driver = 'D1','D2','D3','D4','D5','D6'
condition = 'NORMAL1_SECONDARY','NORMAL2_SECONDARY','NORMAL_MOTORWAY','AGGRESSIVE_MOTORWAY','DROWSY_MOTORWAY','AGGRESSIVE_SECONDARY','DROWSY_SECONDARY'
parameter = 'RAW_GPS','RAW_ACCELEROMETERS','PROC_LANE_DETECTION','PROL_VEHICLE_DETECTION','PROC_OPENSTREETMAP_DATA','SEMANTIC ONLINE'

In [4]:
#FOLDER PATH
D1_NORMAL1_SECONDARY = 'D1/20151110175712-16km-D1-NORMAL1-SECONDARY'
D1_NORMAL2_SECONDARY = 'D1/20151110180824-16km-D1-NORMAL2-SECONDARY'
D1_NORMAL_MOTORWAY = 'D1/20151111123124-25km-D1-NORMAL-MOTORWAY'
D1_AGGRESSIVE_MOTORWAY = 'D1/20151111125233-24km-D1-AGGRESSIVE-MOTORWAY'
D1_DROWSY_MOTORWAY = 'D1/20151111132348-25km-D1-DROWSY-MOTORWAY'
D1_AGGRESSIVE_SECONDARY = 'D1/20151111134545-16km-D1-AGGRESSIVE-SECONDARY'
D1_DROWSY_SECONDARY = 'D1/20151111135612-13km-D1-DROWSY-SECONDARY'
D2_NORMAL1_SECONDARY = 'D2/20151120160904-16km-D2-NORMAL1-SECONDARY'
D2_NORMAL2_SECONDARY = 'D2/20151120162105-17km-D2-NORMAL2-SECONDARY'
D2_NORMAL_MOTORWAY = 'D2/20151120131714-26km-D2-NORMAL-MOTORWAY'
D2_AGGRESSIVE_MOTORWAY = 'D2/20151120133502-26km-D2-AGGRESSIVE-MOTORWAY'
D2_DROWSY_MOTORWAY = 'D2/20151120135152-25km-D2-DROWSY-MOTORWAY'
D2_AGGRESSIVE_SECONDARY = 'D2/20151120163350-16km-D2-AGGRESSIVE-SECONDARY'
D2_DROWSY_SECONDARY = 'D2/20151120164606-16km-D2-DROWSY-SECONDARY'
D3_NORMAL1_SECONDARY = 'D3/20151126124208-16km-D3-NORMAL1-SECONDARY'
D3_NORMAL2_SECONDARY = 'D3/20151126125458-16km-D3-NORMAL2-SECONDARY'
D3_NORMAL_MOTORWAY = 'D3/20151126110502-26km-D3-NORMAL-MOTORWAY'
D3_AGGRESSIVE_MOTORWAY = 'D3/20151126134736-26km-D3-AGGRESSIVE-MOTORWAY'
D3_DROWSY_MOTORWAY = 'D3/20151126113754-26km-D3-DROWSY-MOTORWAY'
D3_AGGRESSIVE_SECONDARY = 'D3/20151126130707-16km-D3-AGGRESSIVE-SECONDARY'
D3_DROWSY_SECONDARY = 'D3/20151126132013-17km-D3-DROWSY-SECONDARY'
D4_NORMAL1_SECONDARY = 'D4/20151203171800-16km-D4-NORMAL1-SECONDARY'
D4_NORMAL2_SECONDARY = 'D4/20151203173103-17km-D4-NORMAL2-SECONDARY'
D4_NORMAL_MOTORWAY = 'D4/20151204152848-25km-D4-NORMAL-MOTORWAY'
D4_AGGRESSIVE_MOTORWAY = 'D4/20151204154908-25km-D4-AGGRESSIVE-MOTORWAY'
D4_DROWSY_MOTORWAY = 'D4/20151204160823-25km-D4-DROWSY-MOTORWAY'
D4_AGGRESSIVE_SECONDARY = 'D4/20151203174324-16km-D4-AGGRESSIVE-SECONDARY'
D4_DROWSY_SECONDARY = 'D4/20151203175637-17km-D4-DROWSY-SECONDARY'
D5_NORMAL1_SECONDARY = 'D5/20151211162829-16km-D5-NORMAL1-SECONDARY'
D5_NORMAL2_SECONDARY = 'D5/20151211164124-17km-D5-NORMAL2-SECONDARY'
D5_NORMAL_MOTORWAY = 'D5/20151209151242-25km-D5-NORMAL-MOTORWAY'
D5_AGGRESSIVE_MOTORWAY = 'D5/20151209153137-25km-D5-AGGRESSIVE-MOTORWAY'
D5_DROWSY_MOTORWAY = 'D5/20151211160213-25km-D5-DROWSY-MOTORWAY'
D5_AGGRESSIVE_SECONDARY = 'D5/20151211165606-12km-D5-AGGRESSIVE-SECONDARY'
D5_DROWSY_SECONDARY = 'D5/20151211170502-16km-D5-DROWSY-SECONDARY'
D6_NORMAL_SECONDARY = 'D6/20151221112434-17km-D6-NORMAL-SECONDARY'
D6_NORMAL_MOTORWAY = 'D6/20151217162714-26km-D6-NORMAL-MOTORWAY'
D6_AGGRESSIVE_MOTORWAY = 'D6/20151221120051-26km-D6-AGGRESSIVE-MOTORWAY'
D6_DROWSY_MOTORWAY = 'D6/20151217164730-25km-D6-DROWSY-MOTORWAY'
D6_DROWSY_SECONDARY = 'D6/20151221113846-16km-D6-DROWSY-SECONDARY'

In [5]:
def RAW_GPS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_GPS = pd.read_csv (driver_condition+'/RAW_GPS.txt', sep=" ").iloc[:,0:-4]
    RAW_GPS.columns = [
        'Timestamp (seconds)', 
        'Speed (km/h)', 
        'Latitude coordinate (degrees)', 
        'Longitude coordinate (degrees)', 
        'Altitude (meters)', 
        'Vertical accuracy (degrees)', 
        'Horizontal accuracy (degrees)', 
        'Course (degrees)', 
        'Difcourse: course variation (degrees)'
    ] 
    
    return RAW_GPS

def RAW_ACCELEROMETERS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_ACCELEROMETERS = pd.read_csv (driver_condition+'/RAW_ACCELEROMETERS.txt', sep=" ").iloc[:,0:-1]
    RAW_ACCELEROMETERS.columns = [
        'Timestamp (seconds)',
        'Boolean of system activated (1 if >50km/h)',
        'Acceleration in X (Gs)',
        'Acceleration in Y (Gs)',
        'Acceleration in Z (Gs)',
        'Acceleration in X filtered by KF (Gs)',
        'Acceleration in Y filtered by KF (Gs)',
        'Acceleration in Z filtered by KF (Gs)',
        'Roll (degrees)',
        'Pitch (degrees)',
        'Yaw (degrees)']
    return RAW_ACCELEROMETERS

def PROC_LANE_DETECTION(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_LANE_DETECTION = pd.read_csv (driver_condition+'/PROC_LANE_DETECTION.txt', sep=" ")
    PROC_LANE_DETECTION.columns = [
        'Timestamp (seconds)',
        'X: car position relative to lane center (meters)',
        'Phi: car angle relative to lane curvature (degrees)',
        'W: road width (meters)',
        "State of the lane det. algorithm [-1=calibrating, 0=initializing, 1=undetected, 2=detected/running]" ]
    return PROC_LANE_DETECTION

def PROC_VEHICLE_DETECTION(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_VEHICLE_DETECTION = pd.read_csv (driver_condition+'/PROC_VEHICLE_DETECTION.txt', sep=" ").iloc[:,0:-1]
    PROC_VEHICLE_DETECTION.columns = [
        'Timestamp (seconds)',
        "Distance to ahead vehicle in current lane (meters) [value -1 means no car is detected in front]",
        "Time of impact to ahead vehicle (seconds) [distance related to own speed]",
        "Number of detected vehicles in this frame (traffic)",
        "GPS speed (km/h) [same as in RAW GPS]" ]    
    return PROC_VEHICLE_DETECTION

def PROC_OPENSTREETMAP_DATA(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_OPENSTREETMAP_DATA = pd.read_csv (driver_condition+'/PROC_OPENSTREETMAP_DATA.txt', sep=" ")
    PROC_OPENSTREETMAP_DATA.columns = [
        'Timestamp (seconds)',
        "Maximum allowed speed of current road (km/h)",
        "Reliability of obtained maxspeed (0=unknown, 1=reliable, 2=used previously obtained maxspeed,3=estimated by type of road)",
        "Type of road (motorway, trunk, secondary...)",
        "Number of lanes in current road",
        "Estimated current lane (1=right lane, 2=first left lane, 3=second left lane, etc) [experimental]",
        "GPS Latitude used to query OSM (degrees)",
        "GPS Longitude used to query OSM (degrees)",
        "OSM delay to answer query (seconds)",
        "GPS speed (km/h) [same as in RAW GPS]]",] 
    del PROC_OPENSTREETMAP_DATA["Type of road (motorway, trunk, secondary...)"]
    return PROC_OPENSTREETMAP_DATA

def SEMANTIC_ONLINE(path, driver_condition):
    
    driver_condition = path+driver_condition

    SEMANTIC_ONLINE = pd.read_csv (driver_condition+'/SEMANTIC_ONLINE.txt', sep=" ").iloc[:,-5:-2]
    SEMANTIC_ONLINE2 = pd.read_csv (driver_condition+'/SEMANTIC_ONLINE.txt', sep=' ').iloc[:,0:1]
    SEMANTIC_ONLINE.columns = [
        'Ratio normal (base 1)',
        'Ratio drowsy (base 1)',
        'Ratio aggressive (base 1)']
    SEMANTIC_ONLINE.insert(0, 'Timestamp (seconds)', SEMANTIC_ONLINE2, True)
    return SEMANTIC_ONLINE

In [6]:
def round_down(n, decimals=0):
    multiplier = 100
    return math.floor(n / multiplier) * multiplier

#split every nth columns
split_columns = 100

In [7]:
D1_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_RAW_GPS, D1_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL1_SECONDARY)
D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL1_SECONDARY)
#D1_NORMAL1_SECONDARY_merged = pd.merge_asof(D1_NORMAL1_SECONDARY_merged, D1_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL1_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D1_NORMAL1_SECONDARY_merged))]

D1_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_RAW_GPS, D1_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL2_SECONDARY)
D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL2_SECONDARY)
#D1_NORMAL2_SECONDARY_merged = pd.merge_asof(D1_NORMAL2_SECONDARY_merged, D1_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL2_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D1_NORMAL2_SECONDARY_merged))]

D1_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_RAW_GPS, D1_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_NORMAL_MOTORWAY)
D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_NORMAL_MOTORWAY)
#D1_NORMAL_MOTORWAY_merged = pd.merge_asof(D1_NORMAL_MOTORWAY_merged, D1_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D1_NORMAL_MOTORWAY_merged))]

D1_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_RAW_GPS, D1_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_AGGRESSIVE_MOTORWAY)
D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_AGGRESSIVE_MOTORWAY)
#D1_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D1_AGGRESSIVE_MOTORWAY_merged, D1_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D1_AGGRESSIVE_MOTORWAY_merged))]

D1_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_RAW_GPS, D1_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_DROWSY_MOTORWAY)
D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_DROWSY_MOTORWAY)
#D1_DROWSY_MOTORWAY_merged = pd.merge_asof(D1_DROWSY_MOTORWAY_merged, D1_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D1_DROWSY_MOTORWAY_merged))]

D1_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_RAW_GPS, D1_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_AGGRESSIVE_SECONDARY)
D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_AGGRESSIVE_SECONDARY)
#D1_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D1_AGGRESSIVE_SECONDARY_merged, D1_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_AGGRESSIVE_SECONDARY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D1_AGGRESSIVE_SECONDARY_merged))]

D1_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_RAW_GPS, D1_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D1_DROWSY_SECONDARY)
D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D1_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D1_DROWSY_SECONDARY)
#D1_DROWSY_SECONDARY_merged = pd.merge_asof(D1_DROWSY_SECONDARY_merged, D1_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D1_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D1_DROWSY_SECONDARY_merged))]

In [8]:
def PROC_OPENSTREETMAP_DATA(path, driver_condition):
    
    driver_condition = path+driver_condition

    PROC_OPENSTREETMAP_DATA = pd.read_csv (driver_condition+'/PROC_OPENSTREETMAP_DATA.txt', sep=" ").iloc[:,0:-1]
    PROC_OPENSTREETMAP_DATA.columns = [
        'Timestamp (seconds)',
        "Maximum allowed speed of current road (km/h)",
        "Reliability of obtained maxspeed (0=unknown, 1=reliable, 2=used previously obtained maxspeed,3=estimated by type of road)",
        "Type of road (motorway, trunk, secondary...)",
        "Number of lanes in current road",
        "Estimated current lane (1=right lane, 2=first left lane, 3=second left lane, etc) [experimental]",
        "GPS Latitude used to query OSM (degrees)",
        "GPS Longitude used to query OSM (degrees)",
        "OSM delay to answer query (seconds)",
        "GPS speed (km/h) [same as in RAW GPS]]"] 
    del PROC_OPENSTREETMAP_DATA["Type of road (motorway, trunk, secondary...)"]
    return PROC_OPENSTREETMAP_DATA


In [9]:
D2_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_RAW_GPS, D2_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL1_SECONDARY)
D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL1_SECONDARY)
#D2_NORMAL1_SECONDARY_merged = pd.merge_asof(D2_NORMAL1_SECONDARY_merged, D2_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL1_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D2_NORMAL1_SECONDARY_merged))]

D2_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_RAW_GPS, D2_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL2_SECONDARY)
D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL2_SECONDARY)
#D2_NORMAL2_SECONDARY_merged = pd.merge_asof(D2_NORMAL2_SECONDARY_merged, D2_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL2_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D2_NORMAL2_SECONDARY_merged))]

D2_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_RAW_GPS, D2_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_NORMAL_MOTORWAY)
D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_NORMAL_MOTORWAY)
#D2_NORMAL_MOTORWAY_merged = pd.merge_asof(D2_NORMAL_MOTORWAY_merged, D2_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D2_NORMAL_MOTORWAY_merged))]

D2_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_RAW_GPS, D2_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_AGGRESSIVE_MOTORWAY)
D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_AGGRESSIVE_MOTORWAY)
#D2_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D2_AGGRESSIVE_MOTORWAY_merged, D2_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D2_AGGRESSIVE_MOTORWAY_merged))]

D2_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_RAW_GPS, D2_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_DROWSY_MOTORWAY)
D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_DROWSY_MOTORWAY)
#D2_DROWSY_MOTORWAY_merged = pd.merge_asof(D2_DROWSY_MOTORWAY_merged, D2_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D2_DROWSY_MOTORWAY_merged))]

D2_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_RAW_GPS, D2_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_AGGRESSIVE_SECONDARY)
D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_AGGRESSIVE_SECONDARY)
#D2_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D2_AGGRESSIVE_SECONDARY_merged, D2_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_AGGRESSIVE_SECONDARY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D2_AGGRESSIVE_SECONDARY_merged))]

D2_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_RAW_GPS, D2_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D2_DROWSY_SECONDARY)
D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D2_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D2_DROWSY_SECONDARY)
#D2_DROWSY_SECONDARY_merged = pd.merge_asof(D2_DROWSY_SECONDARY_merged, D2_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D2_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D2_DROWSY_SECONDARY_merged))]

In [10]:
D3_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_RAW_GPS, D3_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL1_SECONDARY)
D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL1_SECONDARY)
#D3_NORMAL1_SECONDARY_merged = pd.merge_asof(D3_NORMAL1_SECONDARY_merged, D3_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL1_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D3_NORMAL1_SECONDARY_merged))]

D3_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_RAW_GPS, D3_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL2_SECONDARY)
D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL2_SECONDARY)
#D3_NORMAL2_SECONDARY_merged = pd.merge_asof(D3_NORMAL2_SECONDARY_merged, D3_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL2_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D3_NORMAL2_SECONDARY_merged))]

D3_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_RAW_GPS, D3_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_NORMAL_MOTORWAY)
D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_NORMAL_MOTORWAY)
#D3_NORMAL_MOTORWAY_merged = pd.merge_asof(D3_NORMAL_MOTORWAY_merged, D3_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D3_NORMAL_MOTORWAY_merged))]

D3_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_RAW_GPS, D3_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_AGGRESSIVE_MOTORWAY)
D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_AGGRESSIVE_MOTORWAY)
#D3_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D3_AGGRESSIVE_MOTORWAY_merged, D3_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D3_AGGRESSIVE_MOTORWAY_merged))]

D3_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_RAW_GPS, D3_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_DROWSY_MOTORWAY)
D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_DROWSY_MOTORWAY)
#D3_DROWSY_MOTORWAY_merged = pd.merge_asof(D3_DROWSY_MOTORWAY_merged, D3_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D3_DROWSY_MOTORWAY_merged))]

D3_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_RAW_GPS, D3_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_AGGRESSIVE_SECONDARY)
D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_AGGRESSIVE_SECONDARY)
#D3_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D3_AGGRESSIVE_SECONDARY_merged, D3_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_AGGRESSIVE_SECONDARY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D3_AGGRESSIVE_SECONDARY_merged))]

D3_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_RAW_GPS, D3_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D3_DROWSY_SECONDARY)
D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D3_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D3_DROWSY_SECONDARY)
#D3_DROWSY_SECONDARY_merged = pd.merge_asof(D3_DROWSY_SECONDARY_merged, D3_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D3_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D3_DROWSY_SECONDARY_merged))]


In [11]:
D4_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_RAW_GPS, D4_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL1_SECONDARY)
D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL1_SECONDARY)
#D4_NORMAL1_SECONDARY_merged = pd.merge_asof(D4_NORMAL1_SECONDARY_merged, D4_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL1_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D4_NORMAL1_SECONDARY_merged))]

D4_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_RAW_GPS, D4_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL2_SECONDARY)
D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL2_SECONDARY)
#D4_NORMAL2_SECONDARY_merged = pd.merge_asof(D4_NORMAL2_SECONDARY_merged, D4_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL2_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D4_NORMAL2_SECONDARY_merged))]

D4_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_RAW_GPS, D4_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_NORMAL_MOTORWAY)
D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_NORMAL_MOTORWAY)
#D4_NORMAL_MOTORWAY_merged = pd.merge_asof(D4_NORMAL_MOTORWAY_merged, D4_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D4_NORMAL_MOTORWAY_merged))]

D4_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_RAW_GPS, D4_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_AGGRESSIVE_MOTORWAY)
D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_AGGRESSIVE_MOTORWAY)
#D4_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D4_AGGRESSIVE_MOTORWAY_merged, D4_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D4_AGGRESSIVE_MOTORWAY_merged))]

D4_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_RAW_GPS, D4_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_DROWSY_MOTORWAY)
D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_DROWSY_MOTORWAY)
#D4_DROWSY_MOTORWAY_merged = pd.merge_asof(D4_DROWSY_MOTORWAY_merged, D4_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D4_DROWSY_MOTORWAY_merged))]

D4_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_RAW_GPS, D4_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_AGGRESSIVE_SECONDARY)
D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_AGGRESSIVE_SECONDARY)
#D4_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D4_AGGRESSIVE_SECONDARY_merged, D4_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_AGGRESSIVE_SECONDARY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D4_AGGRESSIVE_SECONDARY_merged))]

D4_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_RAW_GPS, D4_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D4_DROWSY_SECONDARY)
D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D4_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D4_DROWSY_SECONDARY)
#D4_DROWSY_SECONDARY_merged = pd.merge_asof(D4_DROWSY_SECONDARY_merged, D4_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D4_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D4_DROWSY_SECONDARY_merged))]

In [12]:
D5_NORMAL1_SECONDARY_RAW_GPS = RAW_GPS(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_RAW_GPS, D5_NORMAL1_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL1_SECONDARY)
D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_NORMAL1_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL1_SECONDARY)
#D5_NORMAL1_SECONDARY_merged = pd.merge_asof(D5_NORMAL1_SECONDARY_merged, D5_NORMAL1_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL1_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D5_NORMAL1_SECONDARY_merged))]

D5_NORMAL2_SECONDARY_RAW_GPS = RAW_GPS(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_RAW_GPS, D5_NORMAL2_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL2_SECONDARY)
D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_NORMAL2_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL2_SECONDARY)
#D5_NORMAL2_SECONDARY_merged = pd.merge_asof(D5_NORMAL2_SECONDARY_merged, D5_NORMAL2_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL2_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D5_NORMAL2_SECONDARY_merged))]

D5_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_RAW_GPS, D5_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_NORMAL_MOTORWAY)
D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_NORMAL_MOTORWAY)
#D5_NORMAL_MOTORWAY_merged = pd.merge_asof(D5_NORMAL_MOTORWAY_merged, D5_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D5_NORMAL_MOTORWAY_merged))]

D5_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_RAW_GPS, D5_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_AGGRESSIVE_MOTORWAY)
D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_AGGRESSIVE_MOTORWAY)
#D5_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D5_AGGRESSIVE_MOTORWAY_merged, D5_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D5_AGGRESSIVE_MOTORWAY_merged))]

D5_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_RAW_GPS, D5_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_DROWSY_MOTORWAY)
D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_DROWSY_MOTORWAY)
#D5_DROWSY_MOTORWAY_merged = pd.merge_asof(D5_DROWSY_MOTORWAY_merged, D5_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D5_DROWSY_MOTORWAY_merged))]

D5_AGGRESSIVE_SECONDARY_RAW_GPS = RAW_GPS(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_RAW_GPS, D5_AGGRESSIVE_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_AGGRESSIVE_SECONDARY)
D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_AGGRESSIVE_SECONDARY)
#D5_AGGRESSIVE_SECONDARY_merged = pd.merge_asof(D5_AGGRESSIVE_SECONDARY_merged, D5_AGGRESSIVE_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_AGGRESSIVE_SECONDARY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D5_AGGRESSIVE_SECONDARY_merged))]

D5_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_RAW_GPS, D5_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D5_DROWSY_SECONDARY)
D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D5_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D5_DROWSY_SECONDARY)
#D5_DROWSY_SECONDARY_merged = pd.merge_asof(D5_DROWSY_SECONDARY_merged, D5_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D5_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D5_DROWSY_SECONDARY_merged))]


In [13]:
D6_NORMAL_SECONDARY_RAW_GPS = RAW_GPS(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_RAW_GPS, D6_NORMAL_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_NORMAL_SECONDARY)
D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D6_NORMAL_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_NORMAL_SECONDARY)
#D6_NORMAL_SECONDARY_merged = pd.merge_asof(D6_NORMAL_SECONDARY_merged, D6_NORMAL_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_SECONDARY_merged['Behavior']=['NORMAL' for i in range(len(D6_NORMAL_SECONDARY_merged))]

D6_AGGRESSIVE_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_RAW_GPS, D6_AGGRESSIVE_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_AGGRESSIVE_MOTORWAY)
D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D6_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_AGGRESSIVE_MOTORWAY)
#D6_AGGRESSIVE_MOTORWAY_merged = pd.merge_asof(D6_AGGRESSIVE_MOTORWAY_merged, D6_AGGRESSIVE_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D6_AGGRESSIVE_MOTORWAY_merged['Behavior']=['AGGRESSIVE' for i in range(len(D6_AGGRESSIVE_MOTORWAY_merged))]

D6_DROWSY_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_RAW_GPS, D6_DROWSY_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_DROWSY_MOTORWAY)
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_DROWSY_MOTORWAY).sort_values(by='Timestamp (seconds)')
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_DROWSY_MOTORWAY)
D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D6_DROWSY_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_DROWSY_MOTORWAY)
#D6_DROWSY_MOTORWAY_merged = pd.merge_asof(D6_DROWSY_MOTORWAY_merged, D6_DROWSY_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_MOTORWAY_merged['Behavior']=['DROWSY' for i in range(len(D6_DROWSY_MOTORWAY_merged))]

D6_DROWSY_SECONDARY_RAW_GPS = RAW_GPS(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_RAW_GPS, D6_DROWSY_SECONDARY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_DROWSY_SECONDARY)
D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D6_DROWSY_SECONDARY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_DROWSY_SECONDARY)
#D6_DROWSY_SECONDARY_merged = pd.merge_asof(D6_DROWSY_SECONDARY_merged, D6_DROWSY_SECONDARY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D6_DROWSY_SECONDARY_merged['Behavior']=['DROWSY' for i in range(len(D6_DROWSY_SECONDARY_merged))]


In [14]:
def RAW_ACCELEROMETERS(path, driver_condition):
    
    driver_condition = path+driver_condition

    RAW_ACCELEROMETERS = pd.read_csv (driver_condition+'/RAW_ACCELEROMETERS.txt', sep=" ")
    RAW_ACCELEROMETERS.columns = [
        'Timestamp (seconds)',
        'Boolean of system activated (1 if >50km/h)',
        'Acceleration in X (Gs)',
        'Acceleration in Y (Gs)',
        'Acceleration in Z (Gs)',
        'Acceleration in X filtered by KF (Gs)',
        'Acceleration in Y filtered by KF (Gs)',
        'Acceleration in Z filtered by KF (Gs)',
        'Roll (degrees)',
        'Pitch (degrees)',
        'Yaw (degrees)']
    return RAW_ACCELEROMETERS

In [15]:
D6_NORMAL_MOTORWAY_RAW_GPS = RAW_GPS(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_RAW_ACCELEROMETERS = RAW_ACCELEROMETERS(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_RAW_GPS, D6_NORMAL_MOTORWAY_RAW_ACCELEROMETERS, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_LANE_DETECTION = PROC_LANE_DETECTION(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_LANE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION = PROC_VEHICLE_DETECTION(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_VEHICLE_DETECTION, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA = PROC_OPENSTREETMAP_DATA(path, D6_NORMAL_MOTORWAY)
D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_PROC_OPENSTREETMAP_DATA, on='Timestamp (seconds)', direction="nearest")
#D6_NORMAL_MOTORWAY_SEMANTIC_ONLINE = SEMANTIC_ONLINE(path, D6_NORMAL_MOTORWAY)
#D6_NORMAL_MOTORWAY_merged = pd.merge_asof(D6_NORMAL_MOTORWAY_merged, D6_NORMAL_MOTORWAY_SEMANTIC_ONLINE, on='Timestamp (seconds)', direction="nearest")
D6_NORMAL_MOTORWAY_merged['Behavior']=['NORMAL' for i in range(len(D6_NORMAL_MOTORWAY_merged))]


In [16]:
#PREPARING INPUT DATA, WITHOUT TRIMMING COLUMNS TO HUNDREDS
#ERASING TIMESTAMPS and the BEHAVIOR

D1_NORMAL1_SECONDARY_trimmed = D1_NORMAL1_SECONDARY_merged.iloc[:(D1_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D1_NORMAL2_SECONDARY_trimmed = D1_NORMAL2_SECONDARY_merged.iloc[:(D1_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D1_NORMAL_MOTORWAY_trimmed = D1_NORMAL_MOTORWAY_merged.iloc[:(D1_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D1_AGGRESSIVE_MOTORWAY_trimmed = D1_AGGRESSIVE_MOTORWAY_merged.iloc[:(D1_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D1_DROWSY_MOTORWAY_trimmed = D1_DROWSY_MOTORWAY_merged.iloc[:(D1_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D1_AGGRESSIVE_SECONDARY_trimmed = D1_AGGRESSIVE_SECONDARY_merged.iloc[:(D1_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D1_DROWSY_SECONDARY_trimmed = D1_DROWSY_SECONDARY_merged.iloc[:(D1_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL1_SECONDARY_trimmed = D2_NORMAL1_SECONDARY_merged.iloc[:(D2_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL2_SECONDARY_trimmed = D2_NORMAL2_SECONDARY_merged.iloc[:(D2_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL_MOTORWAY_trimmed = D2_NORMAL_MOTORWAY_merged.iloc[:(D2_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D2_AGGRESSIVE_MOTORWAY_trimmed = D2_AGGRESSIVE_MOTORWAY_merged.iloc[:(D2_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D2_DROWSY_MOTORWAY_trimmed = D2_DROWSY_MOTORWAY_merged.iloc[:(D2_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D2_AGGRESSIVE_SECONDARY_trimmed = D2_AGGRESSIVE_SECONDARY_merged.iloc[:(D2_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D2_DROWSY_SECONDARY_trimmed = D2_DROWSY_SECONDARY_merged.iloc[:(D2_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL1_SECONDARY_trimmed = D3_NORMAL1_SECONDARY_merged.iloc[:(D3_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL2_SECONDARY_trimmed = D3_NORMAL2_SECONDARY_merged.iloc[:(D3_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL_MOTORWAY_trimmed = D3_NORMAL_MOTORWAY_merged.iloc[:(D3_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D3_AGGRESSIVE_MOTORWAY_trimmed = D3_AGGRESSIVE_MOTORWAY_merged.iloc[:(D3_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D3_DROWSY_MOTORWAY_trimmed = D3_DROWSY_MOTORWAY_merged.iloc[:(D3_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D3_AGGRESSIVE_SECONDARY_trimmed = D3_AGGRESSIVE_SECONDARY_merged.iloc[:(D3_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D3_DROWSY_SECONDARY_trimmed = D3_DROWSY_SECONDARY_merged.iloc[:(D3_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL1_SECONDARY_trimmed = D4_NORMAL1_SECONDARY_merged.iloc[:(D4_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL2_SECONDARY_trimmed = D4_NORMAL2_SECONDARY_merged.iloc[:(D4_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL_MOTORWAY_trimmed = D4_NORMAL_MOTORWAY_merged.iloc[:(D4_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D4_AGGRESSIVE_MOTORWAY_trimmed = D4_AGGRESSIVE_MOTORWAY_merged.iloc[:(D4_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D4_DROWSY_MOTORWAY_trimmed = D4_DROWSY_MOTORWAY_merged.iloc[:(D4_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D4_AGGRESSIVE_SECONDARY_trimmed = D4_AGGRESSIVE_SECONDARY_merged.iloc[:(D4_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D4_DROWSY_SECONDARY_trimmed = D4_DROWSY_SECONDARY_merged.iloc[:(D4_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL1_SECONDARY_trimmed = D5_NORMAL1_SECONDARY_merged.iloc[:(D5_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL2_SECONDARY_trimmed = D5_NORMAL2_SECONDARY_merged.iloc[:(D5_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL_MOTORWAY_trimmed = D5_NORMAL_MOTORWAY_merged.iloc[:(D5_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D5_AGGRESSIVE_MOTORWAY_trimmed = D5_AGGRESSIVE_MOTORWAY_merged.iloc[:(D5_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D5_DROWSY_MOTORWAY_trimmed = D5_DROWSY_MOTORWAY_merged.iloc[:(D5_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D5_AGGRESSIVE_SECONDARY_trimmed = D5_AGGRESSIVE_SECONDARY_merged.iloc[:(D5_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D5_DROWSY_SECONDARY_trimmed = D5_DROWSY_SECONDARY_merged.iloc[:(D5_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D6_NORMAL_SECONDARY_trimmed = D6_NORMAL_SECONDARY_merged.iloc[:(D6_NORMAL_SECONDARY_merged.shape[0]),1:-1]
D6_NORMAL_MOTORWAY_trimmed = D6_NORMAL_MOTORWAY_merged.iloc[:(D6_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D6_AGGRESSIVE_MOTORWAY_trimmed = D6_AGGRESSIVE_MOTORWAY_merged.iloc[:(D6_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D6_DROWSY_MOTORWAY_trimmed = D6_DROWSY_MOTORWAY_merged.iloc[:(D6_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D6_DROWSY_SECONDARY_trimmed = D6_DROWSY_SECONDARY_merged.iloc[:(D6_DROWSY_SECONDARY_merged.shape[0]),1:-1]

In [16]:
#PREPARING INPUT DATA, TRIMMING COLUMNS TO HUNDREDS
#ERASING TIMESTAMPS and the BEHAVIOR

D1_NORMAL1_SECONDARY_trimmed = D1_NORMAL1_SECONDARY_merged.iloc[:round_down(D1_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D1_NORMAL2_SECONDARY_trimmed = D1_NORMAL2_SECONDARY_merged.iloc[:round_down(D1_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D1_NORMAL_MOTORWAY_trimmed = D1_NORMAL_MOTORWAY_merged.iloc[:round_down(D1_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D1_AGGRESSIVE_MOTORWAY_trimmed = D1_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D1_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D1_DROWSY_MOTORWAY_trimmed = D1_DROWSY_MOTORWAY_merged.iloc[:round_down(D1_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D1_AGGRESSIVE_SECONDARY_trimmed = D1_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D1_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D1_DROWSY_SECONDARY_trimmed = D1_DROWSY_SECONDARY_merged.iloc[:round_down(D1_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL1_SECONDARY_trimmed = D2_NORMAL1_SECONDARY_merged.iloc[:round_down(D2_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL2_SECONDARY_trimmed = D2_NORMAL2_SECONDARY_merged.iloc[:round_down(D2_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D2_NORMAL_MOTORWAY_trimmed = D2_NORMAL_MOTORWAY_merged.iloc[:round_down(D2_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D2_AGGRESSIVE_MOTORWAY_trimmed = D2_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D2_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D2_DROWSY_MOTORWAY_trimmed = D2_DROWSY_MOTORWAY_merged.iloc[:round_down(D2_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D2_AGGRESSIVE_SECONDARY_trimmed = D2_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D2_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D2_DROWSY_SECONDARY_trimmed = D2_DROWSY_SECONDARY_merged.iloc[:round_down(D2_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL1_SECONDARY_trimmed = D3_NORMAL1_SECONDARY_merged.iloc[:round_down(D3_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL2_SECONDARY_trimmed = D3_NORMAL2_SECONDARY_merged.iloc[:round_down(D3_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D3_NORMAL_MOTORWAY_trimmed = D3_NORMAL_MOTORWAY_merged.iloc[:round_down(D3_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D3_AGGRESSIVE_MOTORWAY_trimmed = D3_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D3_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D3_DROWSY_MOTORWAY_trimmed = D3_DROWSY_MOTORWAY_merged.iloc[:round_down(D3_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D3_AGGRESSIVE_SECONDARY_trimmed = D3_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D3_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D3_DROWSY_SECONDARY_trimmed = D3_DROWSY_SECONDARY_merged.iloc[:round_down(D3_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL1_SECONDARY_trimmed = D4_NORMAL1_SECONDARY_merged.iloc[:round_down(D4_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL2_SECONDARY_trimmed = D4_NORMAL2_SECONDARY_merged.iloc[:round_down(D4_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D4_NORMAL_MOTORWAY_trimmed = D4_NORMAL_MOTORWAY_merged.iloc[:round_down(D4_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D4_AGGRESSIVE_MOTORWAY_trimmed = D4_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D4_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D4_DROWSY_MOTORWAY_trimmed = D4_DROWSY_MOTORWAY_merged.iloc[:round_down(D4_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D4_AGGRESSIVE_SECONDARY_trimmed = D4_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D4_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D4_DROWSY_SECONDARY_trimmed = D4_DROWSY_SECONDARY_merged.iloc[:round_down(D4_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL1_SECONDARY_trimmed = D5_NORMAL1_SECONDARY_merged.iloc[:round_down(D5_NORMAL1_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL2_SECONDARY_trimmed = D5_NORMAL2_SECONDARY_merged.iloc[:round_down(D5_NORMAL2_SECONDARY_merged.shape[0]),1:-1]
D5_NORMAL_MOTORWAY_trimmed = D5_NORMAL_MOTORWAY_merged.iloc[:round_down(D5_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D5_AGGRESSIVE_MOTORWAY_trimmed = D5_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D5_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D5_DROWSY_MOTORWAY_trimmed = D5_DROWSY_MOTORWAY_merged.iloc[:round_down(D5_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D5_AGGRESSIVE_SECONDARY_trimmed = D5_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D5_AGGRESSIVE_SECONDARY_merged.shape[0]),1:-1]
D5_DROWSY_SECONDARY_trimmed = D5_DROWSY_SECONDARY_merged.iloc[:round_down(D5_DROWSY_SECONDARY_merged.shape[0]),1:-1]
D6_NORMAL_SECONDARY_trimmed = D6_NORMAL_SECONDARY_merged.iloc[:round_down(D6_NORMAL_SECONDARY_merged.shape[0]),1:-1]
D6_NORMAL_MOTORWAY_trimmed = D6_NORMAL_MOTORWAY_merged.iloc[:round_down(D6_NORMAL_MOTORWAY_merged.shape[0]),1:-1]
D6_AGGRESSIVE_MOTORWAY_trimmed = D6_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D6_AGGRESSIVE_MOTORWAY_merged.shape[0]),1:-1]
D6_DROWSY_MOTORWAY_trimmed = D6_DROWSY_MOTORWAY_merged.iloc[:round_down(D6_DROWSY_MOTORWAY_merged.shape[0]),1:-1]
D6_DROWSY_SECONDARY_trimmed = D6_DROWSY_SECONDARY_merged.iloc[:round_down(D6_DROWSY_SECONDARY_merged.shape[0]),1:-1]

In [17]:
#CONCAT BIG DATASET
dataset = pd.concat([
    D1_NORMAL1_SECONDARY_trimmed,
    D1_NORMAL2_SECONDARY_trimmed,
    D1_NORMAL_MOTORWAY_trimmed,
    D1_AGGRESSIVE_MOTORWAY_trimmed,
    D1_DROWSY_MOTORWAY_trimmed,
    D1_AGGRESSIVE_SECONDARY_trimmed,
    D1_DROWSY_SECONDARY_trimmed,
    D2_NORMAL1_SECONDARY_trimmed,
    D2_NORMAL2_SECONDARY_trimmed,
    D2_NORMAL_MOTORWAY_trimmed,
    D2_AGGRESSIVE_MOTORWAY_trimmed,
    D2_DROWSY_MOTORWAY_trimmed,
    D2_AGGRESSIVE_SECONDARY_trimmed,
    D2_DROWSY_SECONDARY_trimmed,
    D3_NORMAL1_SECONDARY_trimmed,
    D3_NORMAL2_SECONDARY_trimmed,
    D3_NORMAL_MOTORWAY_trimmed,
    D3_AGGRESSIVE_MOTORWAY_trimmed,
    D3_DROWSY_MOTORWAY_trimmed,
    D3_AGGRESSIVE_SECONDARY_trimmed,
    D3_DROWSY_SECONDARY_trimmed,
    D4_NORMAL1_SECONDARY_trimmed,
    D4_NORMAL2_SECONDARY_trimmed,
    D4_NORMAL_MOTORWAY_trimmed,
    D4_AGGRESSIVE_MOTORWAY_trimmed,
    D4_DROWSY_MOTORWAY_trimmed,
    D4_AGGRESSIVE_SECONDARY_trimmed,
    D4_DROWSY_SECONDARY_trimmed,
    D5_NORMAL1_SECONDARY_trimmed,
    D5_NORMAL2_SECONDARY_trimmed,
    D5_NORMAL_MOTORWAY_trimmed,
    D5_AGGRESSIVE_MOTORWAY_trimmed,
    D5_DROWSY_MOTORWAY_trimmed,
    D5_AGGRESSIVE_SECONDARY_trimmed,
    D5_DROWSY_SECONDARY_trimmed,
    D6_NORMAL_SECONDARY_trimmed,
    D6_NORMAL_MOTORWAY_trimmed,
    D6_AGGRESSIVE_MOTORWAY_trimmed,
    D6_DROWSY_MOTORWAY_trimmed,
    D6_DROWSY_SECONDARY_trimmed
])

In [18]:
dataset_array = np.array(dataset)

In [19]:
print(dataset_array.shape)

(30929, 34)


In [20]:
X = np.reshape(dataset_array,(int(dataset_array.shape[0]/split_columns),split_columns,dataset_array.shape[1]))
print(X.shape)

(290, 100, 34)


In [20]:
#PREPARING OUTPUT DATA, WITHOUT TRIMMING COLUMNS TO HUNDREDS
#CROPPING ONLY THE 'BEHAVIOR' ROW

D1_NORMAL1_SECONDARY_output = D1_NORMAL1_SECONDARY_merged.iloc[:(D1_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D1_NORMAL2_SECONDARY_output = D1_NORMAL2_SECONDARY_merged.iloc[:(D1_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D1_NORMAL_MOTORWAY_output = D1_NORMAL_MOTORWAY_merged.iloc[:(D1_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D1_AGGRESSIVE_MOTORWAY_output = D1_AGGRESSIVE_MOTORWAY_merged.iloc[:(D1_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D1_DROWSY_MOTORWAY_output = D1_DROWSY_MOTORWAY_merged.iloc[:(D1_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D1_AGGRESSIVE_SECONDARY_output = D1_AGGRESSIVE_SECONDARY_merged.iloc[:(D1_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D1_DROWSY_SECONDARY_output = D1_DROWSY_SECONDARY_merged.iloc[:(D1_DROWSY_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL1_SECONDARY_output = D2_NORMAL1_SECONDARY_merged.iloc[:(D2_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL2_SECONDARY_output = D2_NORMAL2_SECONDARY_merged.iloc[:(D2_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL_MOTORWAY_output = D2_NORMAL_MOTORWAY_merged.iloc[:(D2_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D2_AGGRESSIVE_MOTORWAY_output = D2_AGGRESSIVE_MOTORWAY_merged.iloc[:(D2_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D2_DROWSY_MOTORWAY_output = D2_DROWSY_MOTORWAY_merged.iloc[:(D2_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D2_AGGRESSIVE_SECONDARY_output = D2_AGGRESSIVE_SECONDARY_merged.iloc[:(D2_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D2_DROWSY_SECONDARY_output = D2_DROWSY_SECONDARY_merged.iloc[:(D2_DROWSY_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL1_SECONDARY_output = D3_NORMAL1_SECONDARY_merged.iloc[:(D3_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL2_SECONDARY_output = D3_NORMAL2_SECONDARY_merged.iloc[:(D3_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL_MOTORWAY_output = D3_NORMAL_MOTORWAY_merged.iloc[:(D3_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D3_AGGRESSIVE_MOTORWAY_output = D3_AGGRESSIVE_MOTORWAY_merged.iloc[:(D3_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D3_DROWSY_MOTORWAY_output = D3_DROWSY_MOTORWAY_merged.iloc[:(D3_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D3_AGGRESSIVE_SECONDARY_output = D3_AGGRESSIVE_SECONDARY_merged.iloc[:(D3_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D3_DROWSY_SECONDARY_output = D3_DROWSY_SECONDARY_merged.iloc[:(D3_DROWSY_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL1_SECONDARY_output = D4_NORMAL1_SECONDARY_merged.iloc[:(D4_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL2_SECONDARY_output = D4_NORMAL2_SECONDARY_merged.iloc[:(D4_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL_MOTORWAY_output = D4_NORMAL_MOTORWAY_merged.iloc[:(D4_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D4_AGGRESSIVE_MOTORWAY_output = D4_AGGRESSIVE_MOTORWAY_merged.iloc[:(D4_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D4_DROWSY_MOTORWAY_output = D4_DROWSY_MOTORWAY_merged.iloc[:(D4_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D4_AGGRESSIVE_SECONDARY_output = D4_AGGRESSIVE_SECONDARY_merged.iloc[:(D4_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D4_DROWSY_SECONDARY_output = D4_DROWSY_SECONDARY_merged.iloc[:(D4_DROWSY_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL1_SECONDARY_output = D5_NORMAL1_SECONDARY_merged.iloc[:(D5_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL2_SECONDARY_output = D5_NORMAL2_SECONDARY_merged.iloc[:(D5_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL_MOTORWAY_output = D5_NORMAL_MOTORWAY_merged.iloc[:(D5_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D5_AGGRESSIVE_MOTORWAY_output = D5_AGGRESSIVE_MOTORWAY_merged.iloc[:(D5_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D5_DROWSY_MOTORWAY_output = D5_DROWSY_MOTORWAY_merged.iloc[:(D5_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D5_AGGRESSIVE_SECONDARY_output = D5_AGGRESSIVE_SECONDARY_merged.iloc[:(D5_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D5_DROWSY_SECONDARY_output = D5_DROWSY_SECONDARY_merged.iloc[:(D5_DROWSY_SECONDARY_merged.shape[0]),-1:]
D6_NORMAL_SECONDARY_output = D6_NORMAL_SECONDARY_merged.iloc[:(D6_NORMAL_SECONDARY_merged.shape[0]),-1:]
D6_NORMAL_MOTORWAY_output = D6_NORMAL_MOTORWAY_merged.iloc[:(D6_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D6_AGGRESSIVE_MOTORWAY_output = D6_AGGRESSIVE_MOTORWAY_merged.iloc[:(D6_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D6_DROWSY_MOTORWAY_output = D6_DROWSY_MOTORWAY_merged.iloc[:(D6_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D6_DROWSY_SECONDARY_output = D6_DROWSY_SECONDARY_merged.iloc[:(D6_DROWSY_SECONDARY_merged.shape[0]),-1:]

In [21]:
#PREPARING OUTPUT DATA, TRIMMING COLUMNS TO HUNDREDS
#CROPPING ONLY THE 'BEHAVIOR' ROW

D1_NORMAL1_SECONDARY_output = D1_NORMAL1_SECONDARY_merged.iloc[:round_down(D1_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D1_NORMAL2_SECONDARY_output = D1_NORMAL2_SECONDARY_merged.iloc[:round_down(D1_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D1_NORMAL_MOTORWAY_output = D1_NORMAL_MOTORWAY_merged.iloc[:round_down(D1_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D1_AGGRESSIVE_MOTORWAY_output = D1_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D1_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D1_DROWSY_MOTORWAY_output = D1_DROWSY_MOTORWAY_merged.iloc[:round_down(D1_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D1_AGGRESSIVE_SECONDARY_output = D1_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D1_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D1_DROWSY_SECONDARY_output = D1_DROWSY_SECONDARY_merged.iloc[:round_down(D1_DROWSY_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL1_SECONDARY_output = D2_NORMAL1_SECONDARY_merged.iloc[:round_down(D2_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL2_SECONDARY_output = D2_NORMAL2_SECONDARY_merged.iloc[:round_down(D2_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D2_NORMAL_MOTORWAY_output = D2_NORMAL_MOTORWAY_merged.iloc[:round_down(D2_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D2_AGGRESSIVE_MOTORWAY_output = D2_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D2_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D2_DROWSY_MOTORWAY_output = D2_DROWSY_MOTORWAY_merged.iloc[:round_down(D2_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D2_AGGRESSIVE_SECONDARY_output = D2_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D2_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D2_DROWSY_SECONDARY_output = D2_DROWSY_SECONDARY_merged.iloc[:round_down(D2_DROWSY_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL1_SECONDARY_output = D3_NORMAL1_SECONDARY_merged.iloc[:round_down(D3_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL2_SECONDARY_output = D3_NORMAL2_SECONDARY_merged.iloc[:round_down(D3_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D3_NORMAL_MOTORWAY_output = D3_NORMAL_MOTORWAY_merged.iloc[:round_down(D3_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D3_AGGRESSIVE_MOTORWAY_output = D3_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D3_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D3_DROWSY_MOTORWAY_output = D3_DROWSY_MOTORWAY_merged.iloc[:round_down(D3_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D3_AGGRESSIVE_SECONDARY_output = D3_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D3_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D3_DROWSY_SECONDARY_output = D3_DROWSY_SECONDARY_merged.iloc[:round_down(D3_DROWSY_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL1_SECONDARY_output = D4_NORMAL1_SECONDARY_merged.iloc[:round_down(D4_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL2_SECONDARY_output = D4_NORMAL2_SECONDARY_merged.iloc[:round_down(D4_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D4_NORMAL_MOTORWAY_output = D4_NORMAL_MOTORWAY_merged.iloc[:round_down(D4_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D4_AGGRESSIVE_MOTORWAY_output = D4_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D4_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D4_DROWSY_MOTORWAY_output = D4_DROWSY_MOTORWAY_merged.iloc[:round_down(D4_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D4_AGGRESSIVE_SECONDARY_output = D4_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D4_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D4_DROWSY_SECONDARY_output = D4_DROWSY_SECONDARY_merged.iloc[:round_down(D4_DROWSY_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL1_SECONDARY_output = D5_NORMAL1_SECONDARY_merged.iloc[:round_down(D5_NORMAL1_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL2_SECONDARY_output = D5_NORMAL2_SECONDARY_merged.iloc[:round_down(D5_NORMAL2_SECONDARY_merged.shape[0]),-1:]
D5_NORMAL_MOTORWAY_output = D5_NORMAL_MOTORWAY_merged.iloc[:round_down(D5_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D5_AGGRESSIVE_MOTORWAY_output = D5_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D5_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D5_DROWSY_MOTORWAY_output = D5_DROWSY_MOTORWAY_merged.iloc[:round_down(D5_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D5_AGGRESSIVE_SECONDARY_output = D5_AGGRESSIVE_SECONDARY_merged.iloc[:round_down(D5_AGGRESSIVE_SECONDARY_merged.shape[0]),-1:]
D5_DROWSY_SECONDARY_output = D5_DROWSY_SECONDARY_merged.iloc[:round_down(D5_DROWSY_SECONDARY_merged.shape[0]),-1:]
D6_NORMAL_SECONDARY_output = D6_NORMAL_SECONDARY_merged.iloc[:round_down(D6_NORMAL_SECONDARY_merged.shape[0]),-1:]
D6_NORMAL_MOTORWAY_output = D6_NORMAL_MOTORWAY_merged.iloc[:round_down(D6_NORMAL_MOTORWAY_merged.shape[0]),-1:]
D6_AGGRESSIVE_MOTORWAY_output = D6_AGGRESSIVE_MOTORWAY_merged.iloc[:round_down(D6_AGGRESSIVE_MOTORWAY_merged.shape[0]),-1:]
D6_DROWSY_MOTORWAY_output = D6_DROWSY_MOTORWAY_merged.iloc[:round_down(D6_DROWSY_MOTORWAY_merged.shape[0]),-1:]
D6_DROWSY_SECONDARY_output = D6_DROWSY_SECONDARY_merged.iloc[:round_down(D6_DROWSY_SECONDARY_merged.shape[0]),-1:]

In [21]:
#CONCAT BIG DATASET OUTPUT
output = pd.concat([
    D1_NORMAL1_SECONDARY_output,
    D1_NORMAL2_SECONDARY_output,
    D1_NORMAL_MOTORWAY_output,
    D1_AGGRESSIVE_MOTORWAY_output,
    D1_DROWSY_MOTORWAY_output,
    D1_AGGRESSIVE_SECONDARY_output,
    D1_DROWSY_SECONDARY_output,
    D2_NORMAL1_SECONDARY_output,
    D2_NORMAL2_SECONDARY_output,
    D2_NORMAL_MOTORWAY_output,
    D2_AGGRESSIVE_MOTORWAY_output,
    D2_DROWSY_MOTORWAY_output,
    D2_AGGRESSIVE_SECONDARY_output,
    D2_DROWSY_SECONDARY_output,
    D3_NORMAL1_SECONDARY_output,
    D3_NORMAL2_SECONDARY_output,
    D3_NORMAL_MOTORWAY_output,
    D3_AGGRESSIVE_MOTORWAY_output,
    D3_DROWSY_MOTORWAY_output,
    D3_AGGRESSIVE_SECONDARY_output,
    D3_DROWSY_SECONDARY_output,
    D4_NORMAL1_SECONDARY_output,
    D4_NORMAL2_SECONDARY_output,
    D4_NORMAL_MOTORWAY_output,
    D4_AGGRESSIVE_MOTORWAY_output,
    D4_DROWSY_MOTORWAY_output,
    D4_AGGRESSIVE_SECONDARY_output,
    D4_DROWSY_SECONDARY_output,
    D5_NORMAL1_SECONDARY_output,
    D5_NORMAL2_SECONDARY_output,
    D5_NORMAL_MOTORWAY_output,
    D5_AGGRESSIVE_MOTORWAY_output,
    D5_DROWSY_MOTORWAY_output,
    D5_AGGRESSIVE_SECONDARY_output,
    D5_DROWSY_SECONDARY_output,
    D6_NORMAL_SECONDARY_output,
    D6_NORMAL_MOTORWAY_output,
    D6_AGGRESSIVE_MOTORWAY_output,
    D6_DROWSY_MOTORWAY_output,
    D6_DROWSY_SECONDARY_output
])

In [22]:
output_array = np.array(output)
print(output_array)

[['NORMAL']
 ['NORMAL']
 ['NORMAL']
 ...
 ['DROWSY']
 ['DROWSY']
 ['DROWSY']]


In [24]:
Y = list()

for i in range(int(output_array.shape[0]/split_columns)):
    condition = output_array[(i*split_columns)]    
    behavior = []
    if(condition=='DROWSY'):
        behavior = [0,1,0]
    elif(condition=='AGGRESSIVE'):
        behavior = [0,0,1]
    else:
        behavior = [1,0,0]
    
    Y.append(behavior)

Y = array(Y)
#Y = np.reshape(Y , (int(output_array.shape[0]/split_columns),3))
print(Y.shape)

(290, 3)


In [25]:
#SPLITTING TRAINING AND TEST DATA
X_train = X[:(int(X.shape[0] * 0.8)),:,:]
X_test = X[(int(X.shape[0] * 0.8)):,:,:]
print(X_train.shape)
print(X_test.shape)
Y_train = Y[:(int(Y.shape[0] * 0.8)),:]
Y_test = Y[(int(Y.shape[0] * 0.8)):,:]
print(Y_train.shape)
print(Y_test.shape)

(232, 100, 34)
(58, 100, 34)
(232, 3)
(58, 3)


In [23]:
dataset.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/input2.csv')
output.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/output2.csv')


In [26]:
Y_train = np.asarray(Y_train).astype('float32')

In [27]:
X_train = np.asarray(X_train).astype('float32')

In [46]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(X.shape[1], X.shape[2]))) #100,35
model.add(LSTM(100, activation='relu'))
model.add(Dense(Y.shape[1])) #3
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 100)          54000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 134,703
Trainable params: 134,703
Non-trainable params: 0
_________________________________________________________________


In [54]:
# fit network
model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), verbose=2, shuffle=False)

Epoch 1/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 2/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 3/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 4/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 5/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 6/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 7/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 8/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 9/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 10/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 11/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 12/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 13/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 14/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 15/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 16/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 17/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 18/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 19/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 20/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 21/

Epoch 164/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 165/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 166/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 167/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 168/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 169/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 170/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 171/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 172/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 173/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 174/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 175/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 176/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 177/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 178/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 179/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 180/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 181/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 182/200
8/8 - 1s - loss: nan - val_loss: nan
Epoch 183/200
8/8 - 1s - loss: 

In [55]:
print(X_test[0])
x_input = X_test[0].reshape((1, 100, 34))
print(x_input)

[[75.9      40.466465 -3.485364 ... -3.485161  0.753    75.9     ]
 [76.3      40.466347 -3.485563 ... -3.485364  1.884    78.      ]
 [78.       40.466232 -3.485762 ... -3.485364  1.884    78.      ]
 ...
 [88.9      40.467003 -3.468904 ... -3.470593  6.427    88.1     ]
 [88.1      40.467018 -3.468614 ... -3.470593  6.427    88.1     ]
 [88.3      40.467014 -3.468324 ... -3.468613  1.6      89.6     ]]
[[[75.9      40.466465 -3.485364 ... -3.485161  0.753    75.9     ]
  [76.3      40.466347 -3.485563 ... -3.485364  1.884    78.      ]
  [78.       40.466232 -3.485762 ... -3.485364  1.884    78.      ]
  ...
  [88.9      40.467003 -3.468904 ... -3.470593  6.427    88.1     ]
  [88.1      40.467018 -3.468614 ... -3.470593  6.427    88.1     ]
  [88.3      40.467014 -3.468324 ... -3.468613  1.6      89.6     ]]]


In [56]:
yhat = model.predict(x_input)
print(Y_test[0])
print(yhat)

[0 1 0]
[[nan nan nan]]


In [30]:
D1_merged = pd.concat([D1_NORMAL1_SECONDARY_merged,
D1_NORMAL2_SECONDARY_merged,
D1_NORMAL_MOTORWAY_merged,
D1_AGGRESSIVE_MOTORWAY_merged,
D1_DROWSY_MOTORWAY_merged,
D1_AGGRESSIVE_SECONDARY_merged,
D1_DROWSY_SECONDARY_merged])
#D1_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D1_merged.csv')
print(D1_merged.shape)
#print(D1_DROWSY_SECONDARY_merged.shape)

(4799, 36)


In [31]:
D2_merged = pd.concat([D2_NORMAL1_SECONDARY_merged,
D2_NORMAL2_SECONDARY_merged,
D2_NORMAL_MOTORWAY_merged,
D2_AGGRESSIVE_MOTORWAY_merged,
D2_DROWSY_MOTORWAY_merged,
D2_AGGRESSIVE_SECONDARY_merged,
D2_DROWSY_SECONDARY_merged])
#D2_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D2_merged.csv')
print(D2_merged.shape)

(5267, 36)


In [32]:
D3_merged = pd.concat([D3_NORMAL1_SECONDARY_merged,
D3_NORMAL2_SECONDARY_merged,
D3_NORMAL_MOTORWAY_merged,
D3_AGGRESSIVE_MOTORWAY_merged,
D3_DROWSY_MOTORWAY_merged,
D3_AGGRESSIVE_SECONDARY_merged,
D3_DROWSY_SECONDARY_merged])
#D3_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D3_merged.csv')
print(D3_merged.shape)

(5399, 36)


In [33]:
D4_merged = pd.concat([D4_NORMAL1_SECONDARY_merged,
D4_NORMAL2_SECONDARY_merged,
D4_NORMAL_MOTORWAY_merged,
D4_AGGRESSIVE_MOTORWAY_merged,
D4_DROWSY_MOTORWAY_merged,
D4_AGGRESSIVE_SECONDARY_merged,
D4_DROWSY_SECONDARY_merged])
#D4_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D4_merged.csv')
print(D4_merged.shape)

(5648, 36)


In [34]:
D5_merged = pd.concat([D5_NORMAL1_SECONDARY_merged,
D5_NORMAL2_SECONDARY_merged,
D5_NORMAL_MOTORWAY_merged,
D5_AGGRESSIVE_MOTORWAY_merged,
D5_DROWSY_MOTORWAY_merged,
D5_AGGRESSIVE_SECONDARY_merged,
D5_DROWSY_SECONDARY_merged])
#D5_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D5_merged.csv')
print(D5_merged.shape)

(5292, 36)


In [35]:
D6_merged = pd.concat([D6_NORMAL_SECONDARY_merged,
D6_NORMAL_MOTORWAY_merged,
D6_AGGRESSIVE_MOTORWAY_merged,
D6_DROWSY_MOTORWAY_merged,
D6_DROWSY_SECONDARY_merged])
#D6_merged.to_csv(r'C:/Users/Geraldo Wibowo/Desktop/Year 4/FYP/JUPYTER TEST 1/11_5_2020/D6_merged.csv')
print(D6_merged.shape)

(4524, 36)
